In [1]:
# Library imports
import tensorflow as tf
import rebound
import numpy as np
import datetime
import matplotlib.pyplot as plt

# Aliases
keras = tf.keras

In [2]:
# Local imports
from utils import load_vartbl, save_vartbl, plot_style
from tf_utils import gpu_grow_memory, TimeHistory
from tf_utils import plot_loss_hist, EpochLoss, TimeHistory
from tf_utils import Identity

from orbital_element import OrbitalElementToConfig, ConfigToOrbitalElement, MeanToTrueAnomaly, G_
from orbital_element import make_model_elt_to_cfg, make_model_cfg_to_elt

from jacobi import CartesianToJacobi, JacobiToCartesian

from g3b_data import make_traj_g3b, make_data_g3b, make_datasets_g3b, traj_to_batch
from g3b_data import make_datasets_solar, make_datasets_hard
from g3b_plot import plot_orbit_q, plot_orbit_v, plot_orbit_a, plot_orbit_energy, plot_orbit_element
from g3b import KineticEnergy_G3B, PotentialEnergy_G3B, Momentum_G3B, AngularMomentum_G3B
from g3b import VectorError, EnergyError
from g3b import Motion_G3B, make_physics_model_g3b
from g3b import compile_and_fit
from g3b_model_math import make_position_model_g3b_math, make_model_g3b_math
# from g3b_model_nn import make_position_model_g3b_nn, make_model_g3b_nn

In [3]:
# Grow GPU memory (must be first operation in TF)
gpu_grow_memory()

In [170]:
# Problem orbital elements
a = 1.000
e = 0.0167
# inc = 0.0000
# inc = 0.0001
inc = 1.0E-37
Omega = 0.0000
omega = 1.796642
f = 1.753358
mu = 39.476963

In [171]:
print(f'{omega + f:8.6f}')

3.550000


In [172]:
# Configuration from these elements
qx, qy, qz, vx, vy, vz = OrbitalElementToConfig()([a, e, inc, Omega, omega, f, mu])
print(f'q0 = [{qx:7.4f}, {qy:7.4f}, {qz:7.4f}]')
print(f'v0 = [{vx:7.4f}, {vy:7.4f}, {vz:7.4f}]')

q0 = [-0.9203, -0.3982, -0.0000]
v0 = [ 2.3934, -5.7906, -0.0000]


In [173]:
# Try to recover these elements
a_, e_, inc_, Omega_, omega_, f_, M_, N_ = ConfigToOrbitalElement()([qx, qy, qz, vx, vy, vz, mu])
print(f'Recovered orbital elements:')
print(f'a    ={a_:9.6f}')
print(f'e    ={e_:9.6f}')
print(f'inc  ={inc_:9.6f}')
print(f'Omega={Omega_:9.6f}')
print(f'omega={omega_:9.6f}')
print(f'f    ={f_:9.6f}')

Recovered orbital elements:
a    = 1.000000
e    = 0.016700
inc  = 0.000000
Omega= 0.014221
omega= 1.782421
f    = 1.753358


In [174]:
# Calculate errors
err_a = np.abs(a_ - a)
err_e = np.abs(e_ - e)
err_inc = np.abs(inc_ - inc)
err_Omega = np.abs(Omega_ - Omega)
err_omega = np.abs(omega_ - omega)
err_f = np.abs(f_ - f)

# Print errors
print(f'Errors in orbital elements:')
print(f'a:     {err_a:6.2e}')
print(f'e:     {err_e:6.2e}')
print(f'inc:   {err_inc:6.2e}')
print(f'Omega: {err_Omega:6.2e}')
print(f'omega: {err_omega:6.2e}')
print(f'f:     {err_f:6.2e}')

Errors in orbital elements:
a:     0.00e+00
e:     9.31e-09
inc:   1.00e-37
Omega: 1.42e-02
omega: 1.42e-02
f:     0.00e+00


In [159]:
class ArcCos2(keras.layers.Layer):
    """
    Variant of arc cosine taking three inputs: x, r, and y
    Returns an angle theta such that r * cos(theta) = x and r * sin(theta) matches the sign of y
    Follows function acos2 in rebound tools.c
    """
    def call(self, inputs):
        # Unpack inputs
        x, r, y = inputs
        # Return the arc cosine with the appropriate sign
        # return tf.acos(x / r) * tf.math.sign(y)
        cosine = tf.clip_by_value(x / r, -1.0, 1.0)
        return tf.acos(cosine) * tf.math.sign(y)

    def get_config(self):
        return dict()

In [97]:
# Promote inputs to double precision to minimize roundoff problems
qx = tf.dtypes.cast(qx, dtype=tf.float64, name='qx')
qy = tf.dtypes.cast(qy, dtype=tf.float64, name='qy')
qz = tf.dtypes.cast(qz, dtype=tf.float64, name='qz')
vx = tf.dtypes.cast(vx, dtype=tf.float64, name='vx')
vy = tf.dtypes.cast(vy, dtype=tf.float64, name='vx')
vz = tf.dtypes.cast(vz, dtype=tf.float64, name='vx')
mu = tf.dtypes.cast(mu, dtype=tf.float64, name='mu')

# See rebound library tools.c, reb_tools_particle_to_orbit_err

# The distance from the primary
# r = tf.sqrt(tf.square(qx) + tf.square(qy) + tf.square(qz))
r = tf.sqrt(tf.math.add_n(
        [tf.square(qx) + tf.square(qy) + tf.square(qz)]), 
        name='r')

# The speed and its square
# v2 = tf.square(vx) + tf.square(vy) + tf.square(vz)
v2 = tf.math.add_n(
        [tf.square(vx) + tf.square(vy) + tf.square(vz)], 
        name='v2')
# v = tf.sqrt(v2)

# The speed squared of a circular orbit
v2_circ = mu / r

# The semi-major axis
two = tf.constant(2.0, dtype=tf.float64)
a = -mu / (v2 - two * v2_circ)

# The specific angular momentum vector and its magnitude
# hx = qy*vz - qz*vy
# hy = qz*vx - qx*vz
# hz = qx*vy - qy*vx
# h = tf.sqrt(tf.square(hx) + tf.square(hy) + tf.square(hz))
hx = tf.subtract(qy*vz, qz*vy, name='hx')
hy = tf.subtract(qz*vx, qx*vz, name='hy')
hz = tf.subtract(qx*vy, qy*vx, name='hz')
h = tf.sqrt(tf.math.add_n(
        [tf.square(hx) + tf.square(hy) + tf.square(hz)]), 
        name='h')

# The excess squared speed vs. a circular orbit
# v2_diff = v2 - v2_circ
v2_diff = tf.subtract(v2, v2_circ, name='v2_diff')

# The dot product of v and r; same as r times the radial speed vr
# rvr = (qx * vx + qy*vy + qz*vz)
rvr = tf.add_n([qx*vx, qy*vy, qz*vz], name='rvr')
# The radial speed
# vr = rvr / r
vr = tf.divide(rvr, r, name='vr')
# Inverse of mu
one = tf.constant(1.0, dtype=tf.float64)
mu_inv = one / mu

# Eccentricity vector
ex = mu_inv * (v2_diff * qx - rvr * vx)
ey = mu_inv * (v2_diff * qy - rvr * vy)
ez = mu_inv * (v2_diff * qz - rvr * vz)
# The eccentricity is the magnitude of this vector
# e = tf.sqrt(tf.square(ex) + tf.square(ey) + tf.square(ez))
e = tf.sqrt(tf.math.add_n(
        [tf.square(ex) + tf.square(ey) + tf.square(ez)]),
        name='e')

# The mean motion
N = tf.sqrt(tf.abs(mu / (a*a*a)), name='N')

# The inclination; zero when h points along z axis, i.e. hz = h
# inc = tf.acos(hz / h, name='inc')
inc = ArcCos2(name='inc_fp64')((hz, h, one))

# Vector pointing along the ascending node = zhat cross h
nx = -hy
ny = hx
n = tf.sqrt(tf.square(nx) + tf.square(ny), name='n')

# Longitude of ascending node
# Omega = tf.acos(nx / n) * tf.math.sign(ny)
Omega = ArcCos2(name='Omega_fp64')((nx, n, ny))

# Compute the eccentric anomaly for elliptical orbits (e < 1)
ea = ArcCos2(name='eccentric_anomaly')((one - r / a, e, vr))

# Compute the mean anomaly from the eccentric anomaly using Kepler's equation
M = ea - e * tf.sin(ea)

# Sum of omega + f is always defined in the orbital plane when i != 0
omega_f = ArcCos2(name='omega_plus_f')((nx*qx + ny*qy, n*r, qz))

# The argument of pericenter
omega = ArcCos2(name='omega_fp64')((nx*ex + ny*ey, n*e, ez))

# The true anomaly; may be larger than pi
f = omega_f - omega

# Shift f to the interval [-pi, +pi]
pi = tf.constant(np.pi, dtype=tf.float64)
two_pi = tf.constant(2.0 * np.pi, dtype=tf.float64)
f = tf.math.floormod(f+pi, two_pi) - pi

# Convert the outputs to single precision
a = tf.dtypes.cast(a, dtype=tf.float32, name='a')
e = tf.dtypes.cast(e, dtype=tf.float32, name='e')
inc = tf.dtypes.cast(inc, dtype=tf.float32, name='inc')
Omega = tf.dtypes.cast(Omega, dtype=tf.float32, name='Omega')
omega = tf.dtypes.cast(omega, dtype=tf.float32, name='omega')
f = tf.dtypes.cast(f, dtype=tf.float32, name='f')
M = tf.dtypes.cast(M, dtype=tf.float32, name='M')
N = tf.dtypes.cast(N, dtype=tf.float32, name='N')

In [23]:
r

<tf.Tensor: id=1421, shape=(), dtype=float64, numpy=1.0027613858151179>

In [22]:
v2

<tf.Tensor: id=1427, shape=(), dtype=float64, numpy=39.25954248263389>

In [24]:
v2_circ

<tf.Tensor: id=1428, shape=(), dtype=float64, numpy=39.36825211026961>

In [25]:
a

<tf.Tensor: id=1544, shape=(), dtype=float32, numpy=1.0>

In [29]:
h

<tf.Tensor: id=1448, shape=(), dtype=float64, numpy=6.282193457015907>

In [30]:
hx

<tf.Tensor: id=1436, shape=(), dtype=float64, numpy=-0.0>

In [31]:
hy

<tf.Tensor: id=1439, shape=(), dtype=float64, numpy=0.0>

In [32]:
hz

<tf.Tensor: id=1442, shape=(), dtype=float64, numpy=6.282193457015907>

In [33]:
v2_diff

<tf.Tensor: id=1449, shape=(), dtype=float64, numpy=-0.10870962763571868>

In [34]:
vr

<tf.Tensor: id=1454, shape=(), dtype=float64, numpy=0.10319789213647321>

In [35]:
ex

<tf.Tensor: id=1460, shape=(), dtype=float64, numpy=-0.003739639225928694>

In [36]:
ey

<tf.Tensor: id=1464, shape=(), dtype=float64, numpy=0.016275894998576776>

In [37]:
ez

<tf.Tensor: id=1468, shape=(), dtype=float64, numpy=0.0>

In [26]:
e

<tf.Tensor: id=1545, shape=(), dtype=float32, numpy=0.01669999>

In [27]:
N

<tf.Tensor: id=1551, shape=(), dtype=float32, numpy=6.283069>

In [28]:
inc

<tf.Tensor: id=1546, shape=(), dtype=float32, numpy=0.0>

In [38]:
nx

<tf.Tensor: id=1488, shape=(), dtype=float64, numpy=-0.0>

In [39]:
ny

<tf.Tensor: id=1436, shape=(), dtype=float64, numpy=-0.0>

In [40]:
n

<tf.Tensor: id=1492, shape=(), dtype=float64, numpy=0.0>

In [41]:
Omega

<tf.Tensor: id=1547, shape=(), dtype=float32, numpy=0.0>

In [42]:
ea

<tf.Tensor: id=1510, shape=(), dtype=float64, numpy=1.7369097799088533>

In [43]:
omega_f

<tf.Tensor: id=1525, shape=(), dtype=float64, numpy=0.0>